In [1]:
import numpy as np
import torch
import torch.nn as nn

In [13]:
def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        # nstep 길이에 맞춰 'P' 넣어주기
        for i in range(2):
            seq[i] = seq[i] + 'P'*(n_step - len(seq[i]))

        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(n_class)[input])
        output_batch.append(np.eye(n_class)[output])
        target_batch.append(target) # not one-hot

    # make tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

In [42]:
ib, ob, tb = make_batch()
print('ib size :', ib.shape)
print('ob size :', ob.shape)
print('tb size :', tb.shape)

ib size : torch.Size([6, 5, 29])
ob size : torch.Size([6, 6, 29])
tb size : torch.Size([6, 6])


In [28]:
print('seq_data :', seq_data)
print('n_class :', n_class)

for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P'*(n_step - len(seq[i]))
        print('seq :', seq)

        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        print('input :', [n for n in seq[0]], ' -> ', input)
        print('output :', [n for n in ('S' + seq[1])], ' -> ', output)
        print('target :', [n for n in (seq[1] + 'E')], ' -> ', target, end='\n\n')

        break

seq_data : [['manPP', 'women'], ['black', 'white'], ['kingP', 'queen'], ['girlP', 'boyPP'], ['upPPP', 'downP'], ['highP', 'lowPP']]
n_class : 29
seq : ['manPP', 'women']
input : ['m', 'a', 'n', 'P', 'P']  ->  [15, 3, 16, 2, 2]
-- 2 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
output : ['S', 'w', 'o', 'm', 'e', 'n']  ->  [0, 25, 17, 15, 7, 16]
target : ['w', 'o', 'm', 'e', 'n', 'E']  ->  [25, 17, 15, 7, 16, 1]



In [40]:
print('After ... ', np.eye(29)[[22, 3, 16, 2, 2]])

After ...  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]


In [14]:
def make_testbatch(input_word):
    input_batch, output_batch = [], []

    input_w = input_word + 'P'*(n_step - len(input_word))
    input = [num_dic[n] for n in input_w]
    output = [num_dic[n] for n in 'S'+'P'*n_step]

    # np.eye : 대각행렬
    input_batch = np.eye(n_class)[input]
    output_batch = np.eye(n_class)[output]
    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

In [15]:
x = torch.rand(16, 32, 3)
print('x의 shape : ', x.shape)

y = x.transpose(0,1)
print('transpose 후,', y.shape)

x의 shape :  torch.Size([16, 32, 3])
transpose 후, torch.Size([32, 16, 3])


In [43]:
# # model
class Seq2Seq(nn.Module):
    def __init__(self):
        # 부모 클래스 호출 : (하위 클래스 이름, 객체)
        super(Seq2Seq, self).__init__()  

        # n_class = 29, n_hidden = 128
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        # enc_input = [max_len(=n_step), batch_size, n_class]
        # dec_input = [max_len(=n_step), batch_size, n_class]
        enc_input = enc_input.transpose(0, 1)
        dec_input = dec_input.transpose(0, 1)

        # enc_states : [num_layers(=1)*num_directions(=1), batch_size, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)

        # outputs : [max_len+1(=6), batch_size, num_directions(=1)*n_hidden(=128)]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        # model : [max_len+1(=6), batch_size, n_class]
        model = self.fc(outputs)   
        return model

In [17]:
if __name__=='__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
    num_dic = {n:i for i, n in enumerate(char_arr)}
    seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

    n_class = len(num_dic)
    batch_size = len(seq_data)

    model = Seq2Seq()

    # Cost function
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # batch 생성
    input_batch, output_batch, target_batch = make_batch()

    for epoch in range(5000):
        hidden = torch.zeros(1, batch_size, n_hidden)

        optimizer.zero_grad()

        output = model(input_batch, hidden, output_batch)
        output = output.transpose(0,1)
        loss = 0

        for i in range(0, len(target_batch)):
            loss += criterion(output[i], target_batch[i])

        if (epoch+1)%1000 == 0:
            print('Epoch :', '%04d'%(epoch+1), 'cost =', '{:6f}'.format(loss))

        # 역전파
        loss.backward()
        optimizer.step()

C:\Users\SULJEE~1\AppData\Local\Temp/ipykernel_8084/1848509464.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)


Epoch : 1000 cost = 0.003337
Epoch : 2000 cost = 0.000910
Epoch : 3000 cost = 0.000388
Epoch : 4000 cost = 0.000195
Epoch : 5000 cost = 0.000106


In [18]:
def translate(word):
    input_batch, output_batch = make_testbatch(word)

    hidden = torch.zeros(1,1,n_hidden)
    output = model(input_batch, hidden, output_batch)

    predict = output.data.max(2, keepdim=True)[1]
    decoded = [char_arr[i] for i in predict]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated.replace('P', '')